We'll be training yolov3_tiny using darknet

# **Data Arrangement and Mounting**

Mount Google Drive and Download data folder


In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive"

/content/drive/MyDrive


In [ ]:
#Download data folder
!gdown https://drive.google.com/uc?id=11uZ5Y0i5SMvjdj3Un5mScDY-7lZeTuYp

Downloading...
From: https://drive.google.com/uc?id=11uZ5Y0i5SMvjdj3Un5mScDY-7lZeTuYp
To: /content/drive/MyDrive/ANPR_Assignment.tar.gz
100% 23.0M/23.0M [00:00<00:00, 105MB/s]


In [ ]:
# !tar -xf ANPR_Assignment.tar.gz
%cd VOC2007/

/content/drive/My Drive/VOC2007


In [ ]:
#Rename Images folder()
!mv "/content/drive/My Drive/VOC2007/JPEGImages/" "/content/drive/My Drive/VOC2007/images/"

Remove test images and only keep trainval images

In [ ]:
f = open('ImageSets/Main/test.txt','r')
out = f.readlines() 

In [ ]:
len(out)

169

In [ ]:
for file_name in out:
  if os.path.exists("JPEGImages/"+file_name.replace("\n","")+".jpg"):
    # print("JPEGImages/"+file_name.replace("\n","")+".jpg")
    os.remove("JPEGImages/"+file_name.replace("\n","")+".jpg")
    os.remove("Annotations/"+file_name.replace("\n","")+".xml")

In [ ]:
%cd "/content/drive/MyDrive"

/content/drive/MyDrive


Clone Darknet and make accordingly

In [ ]:
# !git clone https://github.com/AlexeyAB/darknet/

Cloning into 'darknet'...
remote: Enumerating objects: 15316, done.
remote: Total 15316 (delta 0), reused 0 (delta 0), pack-reused 15316
Receiving objects: 100% (15316/15316), 13.72 MiB | 5.50 MiB/s, done.
Resolving deltas: 100% (10408/10408), done.
Checking out files: 100% (2043/2043), done.


In [ ]:
%cd "/content/drive/My Drive/darknet/"

/content/drive/My Drive/darknet


In [ ]:
#Edit Makefile manually to GPU=1 CUDNN=1 CUDNN_HALF=1 OPENCV=1 AVX=0 OPENMP=1 LIBSO=1, then Make
# !make
!chmod +x ./darknet #Give permisions to darknet

# **Data Prepration**

In [ ]:
!mkdir l_detect

In [ ]:
with open("./l_detect/l_detect.names", "a+") as file_object:
    # Move read cursor to the start of file.
    file_object.seek(0)
    file_object.write("license_plate")

Convert VOC annotations to YOLO annotations

In [ ]:
import glob
import sys
import xml.etree.ElementTree as ET


def getImagesInDir(dir_path):
    """Get all images in folder and returns as list.

    Args:
        dir_path ([String]): [Full path to images and annotations]

    """
    image_list = []
    for filename in glob.glob(dir_path + '/JPEGImages/*.'+file_type):
        image_list.append(filename)
    return image_list

def __convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x, y, w, h)

def convert_annotation( dir_path, output_path, image_path):
    """Convert annotations in specified folder iterating through XML file.

    Args:
        dir_path ([String]): [path to images and annotatiosn]
        output_path ([String]): [path to output YOLO annotations]
        image_path ([List]): [List of image files.]
    """
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]
    # print(dir_path + '/Annotations/' + basename_no_ext + '.xml')

    in_file = open(
        dir_path + '/Annotations/' + basename_no_ext + '.xml')
    out_file = open(output_path + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(
            xmlbox.find('xmax').text), float(
                xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = __convert((w, h), b)
        out_file.write(str(cls_id) + ' ' + ' '.join(
            [str(round(a, 4)) for a in bb]) + '\n')


full_dir_path = '/content/drive/MyDrive/VOC2007'  # Full path to main folder
names_path ="/content/drive/MyDrive/darknet/l_detect/l_detect.names"  # Path to names file
classes = open(names_path, 'r').read().split('\n')
print(classes)
file_type = 'jpg'  # File type of image

output_path = full_dir_path + '/JPEGImages/'

# if not os.path.exists(output_path):
#     os.makedirs(output_path)

image_paths = getImagesInDir(full_dir_path)
list_file = open(full_dir_path + '.txt', 'w')

for image_path in image_paths:
    list_file.write(image_path + '\n')
    convert_annotation(
        full_dir_path, output_path, image_path)
list_file.close()

print('Finished processing')

['license_plate']
Finished processing


Arrange train.txt and test.txt

In [ ]:
# arrange data in train.txt and test.txt files
import os
import numpy as np

DATA_DIR = "/content/drive/MyDrive/VOC2007/images" #directory of data (image and txt)
TXT_DIR = "/content/drive/MyDrive/darknet/l_detect/" #directory where you want to generate train and test txt files
print("Data Dir: ", DATA_DIR);
print("TXT Dir: " , TXT_DIR);


images = [image for image in os.listdir(DATA_DIR) if image.split('.')[-1]== 'jpg' and os.path.exists(os.path.join(DATA_DIR,image.replace(".jpg",".txt")))]

print (len(images))

images_train = images[:int(len(images)*0.8)]
images_test = images[int(len(images)*0.8):]
 
with open('{}/train.txt'.format(TXT_DIR), 'a+') as f:
    for image in images_train:
        print("Processing Training..: " + image);
        f.write('{}\n'.format(os.path.join(DATA_DIR , image)))
 
with open('{}/test.txt'.format(TXT_DIR), 'a+') as f:
    for image in images_test:
        print("Processing Valid..: " + image);
        f.write('{}\n'.format(os.path.join(DATA_DIR , image)))

Data Dir:  /content/drive/MyDrive/VOC2007/images
TXT Dir:  /content/drive/MyDrive/darknet/l_detect/
500
Processing Training..: car_187.jpg
Processing Training..: car_340.jpg
Processing Training..: car_206.jpg
Processing Training..: car_359.jpg
Processing Training..: car_624.jpg
Processing Training..: car_376.jpg
Processing Training..: car_641.jpg
Processing Training..: car_488.jpg
Processing Training..: car_507.jpg
Processing Training..: car_27.jpg
Processing Training..: car_106.jpg
Processing Training..: car_393.jpg
Processing Training..: car_259.jpg
Processing Training..: car_524.jpg
Processing Training..: car_44.jpg
Processing Training..: car_123.jpg
Processing Training..: car_276.jpg
Processing Training..: car_541.jpg
Processing Training..: car_407.jpg
Processing Training..: car_61.jpg
Processing Training..: car_140.jpg
Processing Training..: car_159.jpg
Processing Training..: car_293.jpg
Processing Training..: car_312.jpg
Processing Training..: car_424.jpg
Processing Training..: c

Arrange .data file

In [ ]:
with open("/content/drive/MyDrive/darknet/l_detect/l_detect.data", "a+") as file_object:
    # Move read cursor to the start of file.
    file_object.seek(0)
    file_object.write("train = "+TXT_DIR+"train.txt")
    file_object.write("\n")
    file_object.write("valid = "+TXT_DIR+"test.txt")
    file_object.write("\n")
    file_object.write("names = "+TXT_DIR+"l_detect.names")
    file_object.write("\n")
    file_object.write("backup = "+TXT_DIR+"weights_backup/")

# **Finally Train!**

In [ ]:
!./darknet detector train ./l_detect/l_detect.data ./l_detect/yolov3-tiny.cfg -map -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 408983, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.809277), count: 4, class_loss = 0.023603, iou_loss = 0.144596, total_loss = 0.168198 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 408987, rewritten_bbox = 0.000000 % 

 (next mAP calculation at 12600 iterations) 
 Last accuracy mAP@0.50 = 81.88 %, best = 91.16 % 
 12598: 0.157454, 0.196102 avg loss, 0.001000 rate, 1.229119 seconds, 403136 images, 113.339075 hours left
Loaded: 0.000049 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.778055), count: 4, class_loss = 0.029600, iou_loss = 0.119260, total_loss = 0.148859 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.619644), count: 1, class_loss = 